# Problem 3
# Shopping Mall Revenue Prediction

## Importing required libraries

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

c:\users\naveen\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\naveen\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
c:\users\naveen\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)


### Reading data from given url 

In [2]:
url ="https://raw.githubusercontent.com/edyoda/data-science-complete-tutorial/master/Data/Shopping_Revenue.csv"
df = pd.read_csv(url)
df.head(5)

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,7/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753
1,1,2/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131
2,2,3/9/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379
3,3,2/2/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511
4,4,5/9/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715


### shape of the data ( no.of columns and no.of rows)

In [3]:
df.shape

(137, 43)

### checking empty values are present or not

In [4]:
df.isnull().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            4
P7            4
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

### Dealing with Missing or empty values

In [5]:
missed_data_columns = []
for i in df.columns:
    if df[i].isnull().sum() > 0:
        missed_data_columns.append(i)
missed_data_columns

['P6', 'P7']

 Missing data is so less i am dropping the missing data

In [6]:
for i in missed_data_columns:
    df[i].fillna(method='ffill',inplace=True)

In [7]:
df.isnull().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

### Drop use less columns

In [8]:
df.drop(['Id','Open Date','City'],axis=1,inplace=True)

In [9]:
df.head()

,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,Big Cities,IL,4,5.0,4.0,4.0,2,2.0,5.0,4,...,3.0,5,3,4,5,5,4,3,4,5653753
1,Big Cities,FC,4,5.0,4.0,4.0,1,2.0,5.0,5,...,3.0,0,0,0,0,0,0,0,0,6923131
2,Other,IL,2,4.0,2.0,5.0,2,3.0,5.0,5,...,3.0,0,0,0,0,0,0,0,0,2055379
3,Other,IL,6,4.5,6.0,6.0,4,4.0,10.0,8,...,7.5,25,12,10,6,18,12,12,6,2675511
4,Other,IL,3,4.0,3.0,4.0,2,2.0,5.0,5,...,3.0,5,1,3,2,3,4,3,3,4316715


### Encoding categorical columns

In [10]:
df['City Group'] = df['City Group'].map({'Big Cities':0,'Other':1})
df['Type'] = df['Type'].map({'IL':0, 'FC':1, 'DT':2})

In [11]:
df.head()

,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,0,4,5.0,4.0,4.0,2,2.0,5.0,4,...,3.0,5,3,4,5,5,4,3,4,5653753
1,0,1,4,5.0,4.0,4.0,1,2.0,5.0,5,...,3.0,0,0,0,0,0,0,0,0,6923131
2,1,0,2,4.0,2.0,5.0,2,3.0,5.0,5,...,3.0,0,0,0,0,0,0,0,0,2055379
3,1,0,6,4.5,6.0,6.0,4,4.0,10.0,8,...,7.5,25,12,10,6,18,12,12,6,2675511
4,1,0,3,4.0,3.0,4.0,2,2.0,5.0,5,...,3.0,5,1,3,2,3,4,3,3,4316715


### Datatypes of each column

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City Group  137 non-null    int64  
 1   Type        137 non-null    int64  
 2   P1          137 non-null    int64  
 3   P2          137 non-null    float64
 4   P3          137 non-null    float64
 5   P4          137 non-null    float64
 6   P5          137 non-null    int64  
 7   P6          137 non-null    float64
 8   P7          137 non-null    float64
 9   P8          137 non-null    int64  
 10  P9          137 non-null    int64  
 11  P10         137 non-null    int64  
 12  P11         137 non-null    int64  
 13  P12         137 non-null    int64  
 14  P13         137 non-null    float64
 15  P14         137 non-null    int64  
 16  P15         137 non-null    int64  
 17  P16         137 non-null    int64  
 18  P17         137 non-null    int64  
 19  P18         137 non-null    i

In [13]:
df.describe()

,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
count,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,...,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,1.370000e+02
mean,0.430657,0.569343,4.014599,4.408759,4.317518,4.372263,2.007299,3.313869,5.386861,5.153285,...,3.135036,2.729927,1.941606,2.525547,1.138686,2.489051,2.029197,2.211679,1.116788,4.453533e+06
std,0.496985,0.511567,2.910391,1.514900,1.032337,1.016462,1.209620,2.060667,2.263028,1.858567,...,1.680887,5.536647,3.512093,5.230117,1.698540,5.165093,3.436272,4.168211,1.790768,2.576072e+06
min,0.000000,0.000000,1.000000,1.000000,0.000000,3.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.149870e+06
25%,0.000000,0.000000,2.000000,4.000000,4.000000,4.000000,1.000000,2.000000,5.000000,4.000000,...,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.999068e+06
50%,0.000000,1.000000,3.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,5.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.939804e+06
75%,1.000000,1.000000,4.000000,5.000000,5.000000,5.000000,2.000000,4.000000,5.000000,5.000000,...,3.000000,4.000000,3.000000,3.000000,2.000000,3.000000,4.000000,3.000000,2.000000,5.166635e+06
max,1.000000,2.000000,12.000000,7.500000,7.500000,7.500000,8.000000,10.000000,10.000000,10.000000,...,7.500000,25.000000,15.000000,25.000000,6.000000,24.000000,15.000000,20.000000,8.000000,1.969694e+07


### variance of each column

In [14]:
for i in df.columns:
    print(np.var(df[i]))

0.24519153924023648
0.25979007938622195
8.408546006713198
2.2781714529277
1.0579412861633544
1.0256540039426711
1.4525014651819488
4.215355106825084
5.083914966167617
3.4290585539986145
3.34189354787149
3.3885662528637646
3.6243806276306674
3.742554211732111
1.0665459001545101
7.396238478341947
5.711652192444989
12.200969684053492
4.093558527358943
10.814108370184877
31.18077681282966
13.64920880174756
4.182535031168416
1.5035430763492994
20.638286536309877
5.270179551387925
4.517235867654111
6.772870158239649
4.241461985188343
5.291677766529916
2.8047578453833446
30.430710213650165
12.244765304491448
27.15445681709202
2.863977835793063
26.483456763812672
11.721775267728704
17.24716287495338
3.1834407800095903
6587705852592.529


### finding corelation between columns

In [15]:
df.corr()

,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
City Group,1.000000,0.098585,-0.258556,-0.421095,-0.082172,-0.174134,0.019195,-0.398607,-0.181911,-0.071993,...,0.035497,-0.123097,-0.204542,-0.169750,-0.106115,-0.117024,-0.127973,-0.136617,-0.089976,-0.237403
Type,0.098585,1.000000,-0.198231,-0.150700,-0.247365,-0.057078,-0.006765,-0.094037,-0.198006,-0.123399,...,-0.094343,-0.539807,-0.599333,-0.524885,-0.700818,-0.520788,-0.641135,-0.574185,-0.634963,0.090532
P1,-0.258556,-0.198231,1.000000,0.836671,0.695928,0.677941,0.302820,0.695617,0.821923,0.760820,...,0.586532,0.487589,0.479175,0.552593,0.329795,0.570835,0.449182,0.554951,0.277601,0.070217
P2,-0.421095,-0.150700,0.836671,1.000000,0.473551,0.540322,0.203003,0.634606,0.748183,0.505116,...,0.322511,0.342005,0.353477,0.389376,0.234991,0.394319,0.314798,0.387938,0.161162,0.191518
P3,-0.082172,-0.247365,0.695928,0.473551,1.000000,0.443606,0.168891,0.538678,0.634737,0.689173,...,0.571525,0.473089,0.406701,0.463217,0.312269,0.501575,0.428504,0.484942,0.284065,-0.024613
P4,-0.174134,-0.057078,0.677941,0.540322,0.443606,1.000000,0.715406,0.575687,0.684922,0.654594,...,0.452364,0.387747,0.333626,0.440795,0.182820,0.434245,0.277902,0.411664,0.177916,0.035685
P5,0.019195,-0.006765,0.302820,0.203003,0.168891,0.715406,1.000000,0.347160,0.426051,0.447577,...,0.306904,0.244032,0.112603,0.215568,0.071079,0.243040,0.173309,0.193652,0.033548,-0.028191
P6,-0.398607,-0.094037,0.695617,0.634606,0.538678,0.575687,0.347160,1.000000,0.711690,0.663145,...,0.513073,0.461839,0.384561,0.438959,0.292083,0.482183,0.416134,0.445063,0.288879,0.154820
P7,-0.181911,-0.198006,0.821923,0.748183,0.634737,0.684922,0.426051,0.711690,1.000000,0.737526,...,0.574765,0.524825,0.468206,0.546782,0.355132,0.578787,0.474147,0.560297,0.324432,0.048784
P8,-0.071993,-0.123399,0.760820,0.505116,0.689173,0.654594,0.447577,0.663145,0.737526,1.000000,...,0.754736,0.454937,0.420425,0.516617,0.256416,0.524474,0.405709,0.499777,0.308294,-0.084215


### Scaling

In [16]:
from sklearn.preprocessing import StandardScaler
values = df.values
ss = StandardScaler()
y_scaled = ss.fit_transform(values[:,39].reshape(-1,1))

df['revenue'] = y_scaled

### features and target

In [17]:
x = df.drop('revenue',axis=1)
y = df['revenue']

### spliting the data in x_test,x_train,y_test,y_train

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(109, 39) (28, 39) (109,) (28,)


## Model building

### Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression  
regressor= LinearRegression()  
regressor.fit(x_train, y_train)

y_pred = regressor.predict(x_test)

from sklearn.metrics import r2_score
rank = r2_score(y_pred,y_test)
rank

-3.765333310355593

### Decision Tree

In [20]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(random_state = 0)
reg.fit(x_train, y_train)

y_pred = reg.predict(x_test)

from sklearn.metrics import r2_score
rank = r2_score(y_pred,y_test)
rank

-1.6489917624861463

### Random forest

In [21]:
from sklearn.ensemble import RandomForestRegressor 
re= LinearRegression()  
re.fit(x_train, y_train)

y_pred = re.predict(x_test)

from sklearn.metrics import r2_score
r = r2_score(y_pred,y_test)
r

-3.765333310355593

### The data is not effficient to do model building . The all ML algo will fail if data is a garbage